In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# Teaching a quadruped to walk

Time to try out the learning algorithms that you just implemented on a more difficult problem. The WalkerEnv implements a quadruped robot kind-of thing, see for yourself. The goal is to move in the $x$ direction as fast and as far as possible.

Your goal is to implement a class `WalkerPolicy` with function `determine_actions()` just like the StochasticPolicy we used earlier to control the pendulum. Below is a template of this class, but feel free to alter it however you want. The only important thing is the `determine_actions()` function!

After you implement it, copy `WalkerPolicy` into a separate file `WalkerPolicy.py` that you will upload to BRUTE together with the (optional) learned weights in a zip file. How the policy is implemented is up to you! You are constrained to only the libraries we used so far though, such as torch, numpy etc..

You will get some free points just for uploading a working policy (irrelevant of the performance). Further 2 points will be awarded for successfully traversing a small distance in the x direction.


# Hints

There is no single easy way of doing this, but here are some suggestions on what you could try to improve your policy:

1. This problem is much more difficult, than balancing a pendulum. It is a good idea to use a bit larger network than for the pendulum policy.

2. You can also try to use a different optimizer, such as Adam and play with the hyperparameters.

3. Using a neural network to compute the normal distribution scale $\sigma$ can lead to too much randomness in the actions (i.e. exploration). You can use a fixed $\sigma$ instead, or replace it with a learnable `torch.Parameter` initialized to some small constant. Make sure, you run it through an exponential, or softplus function to ensure $\sigma$ is positive.

4. The exploration can also be reduced by penalizing the variance of the action distribution in an additional loss term.

5. If you see some undesirable behaviour, you can tweak the reward function to penalize it. Even though the $x$ distance is all we care about, adding extra terms to the reward can help guide the learning process (This is known as reward shaping). Simply define a reward function mapping the state $s_{t+1}$ and action $a_t$ to a scalar reward $r_t$ and put it in the config dictionary under the key `'reward_fcn'`. See the `WalkerEnv` class for the implementation of the default reward.

6. Using the normal distribution on a bounded action space can lead to certain problems caused by action clipping. This can be mitigated by using a different distribution, such as the Beta distribution. See the `torch.distributions.beta` module for more information. (Note that Beta distribution is defined on the interval [0,1] and works better with parameters $\alpha,\beta \geq 1$.)


In [2]:
# If you cannot run with the visualization, you can set this to False
VISUALIZE = True
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### import self-made modules

In [3]:
from environment.WalkerEnv import WalkerEnv
from WalkerPolicy import WalkerPolicy
import solution

### Define reward function

In [4]:
DISTANCE_MULTIPLIER = 0
VELOCITY_MULTIPLIER = 1
ACTION_PENALTY_MULTIPLIER = 0.001
STABILITY_PENALTY_MULTIPLIER = 0.001

def walker_reward(state, action):
    pos = state[:15]  # first 15 elements of state vector are generalized coordinates [xyz, quat, joint_angles]
    vel = state[15:]  # last 14 elements of state vector are generalized velocities [xyz_vel, omega, joint_velocities]
    x_velocity = vel[0]  # this is the x axis velocity
    x_distance = pos[0]  # this is the x axis position
    stability_penalty = np.sum(np.abs(vel[1:3]))  # Penalize y and z velocities
    action_penalty = np.sum(np.square(action))  # Penalize large actions
    return (x_distance * DISTANCE_MULTIPLIER + 
            x_velocity * VELOCITY_MULTIPLIER - 
            STABILITY_PENALTY_MULTIPLIER * stability_penalty - 
            ACTION_PENALTY_MULTIPLIER * action_penalty)

## Train loop
### helper functions

In [5]:
def sample_trajectories(env, policy, T, device='cpu'):
    """
    Collect T steps of experience from N parallel walkers in 'env'.
    Returns:
      states:  [T+1, N, state_dim]
      actions: [T,   N, action_dim]
      rewards: [T,   N]
    """
    # 1) Reset environment
    obs_list = env.reset()  # list of length N, each shape(29,)
    obs_list = obs_list[0]
    N = len(obs_list)
    state_dim = len(obs_list[0])
    action_dim = 8  # for your quadruped

    # Convert to torch
    obs = torch.tensor(np.stack(obs_list), dtype=torch.float32, device=device)  # [N, 29]

    # 2) Allocate buffers
    states  = torch.zeros((T+1, N, state_dim),  device=device)
    actions = torch.zeros((T,   N, action_dim), device=device)
    rewards = torch.zeros((T,   N),            device=device)

    states[0] = obs

    # 3) Roll out for T steps
    for t in range(T):
        with torch.no_grad():
            # forward returns (actions, log_probs, state_values, exploration_var)
            a, _, _, _ = policy(states[t])  # shape [N, action_dim]

        actions[t] = a

        # We must flatten if 'env.vector_step' expects shape (N*action_dim,)
        a_np = a.cpu().numpy().reshape(-1)  # shape (N*action_dim,)

        # Step environment
        next_obs_list, reward_list = env.vector_step(a_np)
        # next_obs_list: list of length N, each shape(29,)
        # reward_list:   list of length N

        # convert to torch
        next_obs = torch.tensor(np.stack(next_obs_list), dtype=torch.float32, device=device)
        r = torch.tensor(reward_list, dtype=torch.float32, device=device)

        # store
        states[t+1] = next_obs
        rewards[t]  = r
        obs = next_obs

    return states, actions, rewards

def compute_gae_no_done(rewards, values, gamma=0.99, lam=0.95):
    """
    rewards: [T, N]
    values:  [T+1, N]
    Returns:
      advantages:   [T, N]
      value_target: [T, N] = advantages + values[:T]
    """
    T, N = rewards.shape
    advantages = torch.zeros((T, N), dtype=torch.float32, device=rewards.device)
    last_gae = torch.zeros((N,), dtype=torch.float32, device=rewards.device)

    for t in reversed(range(T)):
        delta = rewards[t] + gamma*values[t+1] - values[t]
        last_gae = delta + gamma*lam*last_gae
        advantages[t] = last_gae

    value_target = advantages + values[:-1]  # shape [T, N]
    return advantages, value_target

def ppo_loss(p_ratios, advantages, epsilon=0.2):
    """
    p_ratios:   exp(log_pi(a|s) - log_pi_old(a|s)) [T*N]
    advantages: [T*N]
    """
    clipped = torch.clamp(p_ratios, 1.0 - epsilon, 1.0 + epsilon) * advantages
    return -torch.min(p_ratios * advantages, clipped).mean()

def value_loss(value_preds, value_targets):
    """
    Simple MSE on the value function
    """
    return 0.5 * (value_preds - value_targets).pow(2).mean()


### Training function

In [6]:
def run_ppo_training():
    # Hyperparameters
    N = 4         # number of parallel quadrupeds
    T = 256         # horizon
    epochs = 300    # training epochs
    gamma = 0.95
    lam = 0.97
    epsilon = 0.2
    sgd_iters = 5
    lr = 1e-3       # Adam learning rate

    best_reward = 0

    # 1) Create environment
    config = {
        'N': N,
        'vis': False,
        'track': 0,
        'reward_fcn': walker_reward
    }
    env = WalkerEnv(config)

    # 2) Create bigger policy
    state_dim = 29
    action_dim = 8
    policy = WalkerPolicy(
        state_dim=state_dim,
        action_dim=action_dim,
    ).to(device)

    optimizer = optim.Adam(policy.parameters(), lr=lr)

    # 3) For logging
    mean_rewards_list = []
    policy_loss_list  = []
    value_loss_list   = []

    # 4) Main training loop with tqdm
    pbar = tqdm(range(epochs), desc='Training PPO')
    for epoch in pbar:
        # (a) Collect rollouts
        states, actions, rewards = sample_trajectories(env, policy, T, device=device)
        # shapes:
        #   states:  [T+1, N, 29]
        #   actions: [T,   N, 8]
        #   rewards: [T,   N]

        with torch.no_grad():
            # Evaluate old log-probs and values
            T_plus_1, N_ = states.shape[0], states.shape[1]
            all_values = policy.value_estimates(states.view(-1, state_dim))  # -> [(T+1)*N, 1]
            all_values = all_values.view(T_plus_1, N_)
            
            states_flat  = states[:-1].reshape(-1, state_dim)  # [T*N, state_dim]
            actions_flat = actions.reshape(-1, action_dim)      # [T*N, action_dim]
            logp_old = policy.log_prob(actions_flat, states_flat)  # [T*N]

        # (b) GAE
        advantages, value_targets = compute_gae_no_done(
            rewards, all_values, gamma=gamma, lam=lam
        )
        # Flatten
        adv_flat = advantages.view(-1)
        val_targ_flat = value_targets.view(-1)

        # Normalize advantages
        adv_mean = adv_flat.mean()
        adv_std  = adv_flat.std() + 1e-8
        adv_flat = (adv_flat - adv_mean) / adv_std

        # (c) PPO update
        for _ in range(sgd_iters):
            logp = policy.log_prob(actions_flat, states_flat)  # [T*N]
            p_ratios = torch.exp(logp - logp_old)
            L_ppo = ppo_loss(p_ratios, adv_flat, epsilon=epsilon)

            # Value
            new_values = policy.value_estimates(states.view(-1, state_dim)).view(T_plus_1, N_)
            new_values_t = new_values[:-1].reshape(-1)
            L_v = value_loss(new_values_t, val_targ_flat)

            # (Optional) Exploration penalty
            # E.g. measure variance of the Beta distribution on states_flat
            exploration_penalty_coef = 0.01
            alpha_beta = policy.actor_network(states_flat)
            alpha, beta = torch.chunk(alpha_beta, 2, dim=-1)
            alpha = F.softplus(alpha) + 1.0
            beta  = F.softplus(beta)  + 1.0
            var_each_dim = (alpha * beta) / ((alpha + beta)**2 * (alpha + beta + 1.0))
            exploration_var = var_each_dim.mean()
            L_exploration = exploration_penalty_coef * exploration_var

            total_loss = L_ppo + L_v + L_exploration

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

        # (d) Logging for each epoch
        ep_mean_reward = rewards.mean().item()
        if ep_mean_reward > best_reward:
            best_reward = ep_mean_reward
            policy.save_weights()
            print(f"New best reward: {best_reward}, saved the model")
        mean_rewards_list.append(ep_mean_reward)
        policy_loss_list.append(L_ppo.item())
        value_loss_list.append(L_v.item())

        # Update tqdm progress bar
        pbar.set_postfix({
            'Epoch': epoch, 
            'MeanReward': f'{ep_mean_reward:.3f}', 
            'PolicyLoss': f'{L_ppo.item():.3f}', 
            'ValueLoss': f'{L_v.item():.3f}'
        })

    # 5) End of training
    #policy.save_weights()
    env.close()

    # 6) Plot results
    plt.figure(figsize=(12,5))

    plt.subplot(1,3,1)
    plt.title("Mean Rewards")
    plt.plot(mean_rewards_list, label='Reward')
    plt.xlabel('Epoch')
    plt.ylabel('Reward')
    plt.legend()

    plt.subplot(1,3,2)
    plt.title("Policy Loss")
    plt.plot(policy_loss_list, label='Policy Loss', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1,3,3)
    plt.title("Value Loss")
    plt.plot(value_loss_list, label='Value Loss', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return mean_rewards_list, policy_loss_list, value_loss_list


### Train model

In [7]:
mean_rewards, p_losses, v_losses = run_ppo_training()


Environment ready


Training PPO:  12%|█▏        | 37/300 [00:33<03:02,  1.44it/s, Epoch=36, MeanReward=0.013, PolicyLoss=-0.008, ValueLoss=0.734] 

New best reward: 0.013285979628562927, saved the model


Training PPO:  14%|█▎        | 41/300 [00:37<03:57,  1.09it/s, Epoch=40, MeanReward=0.021, PolicyLoss=-0.013, ValueLoss=1.003] 

New best reward: 0.021214595064520836, saved the model


Training PPO:  23%|██▎       | 70/300 [01:03<03:30,  1.09it/s, Epoch=69, MeanReward=-0.053, PolicyLoss=-0.013, ValueLoss=10.301]


KeyboardInterrupt: 

## Visualise the trained quadruped

In [8]:
# This is the configuration for the Walker environment
# N is the number of robots controlled in parallel
# vis is a boolean flag to enable visualization
# !! IMPORTANT track is an integer index to enable camera tracking of a particular robot (indexed by the value of the argument), this is useful when evaluating the performance of the policy after training
# reward_fcn is the reward function that the environment will use to calculate the reward
T = 1000
x = -1000
N = 1

# Suppose you have N=4
env = WalkerEnv({'N': N, 'vis': VISUALIZE, "track": 0, "reward_fcn": walker_reward})
obs = env.vector_reset()  # shape (4, 29)
policy = WalkerPolicy()
policy.load_weights()

for i in range(T):
    # obs is now shape (4, 29)
    obs_tensor = torch.tensor(obs, dtype=torch.float32)  # shape (4, 29)
    actions_tensor = policy.determine_actions(obs_tensor)  # shape (4, 8)
    # Flatten to (32,) for env.vector_step
    actions_np = actions_tensor.cpu().numpy().reshape(-1)
    obs, reward = env.vector_step(actions_np)  # returns new obs, reward for all 4 walkers

    # For tracking the maximum x across all walkers:
    # obs is a list of length 4, each a (29,)-shaped array
    for w in range(N):
        x = max(x, obs[w][0])

env.close()
print(f"After {T} steps, the maximum x value reached was {x}")


Environment ready


/home/pilcsimo/CVUT/UROB/hw5-rl/WalkerPolicy.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path))
/tmp/ipykernel_15309/788223384.py:1

After 1000 steps, the maximum x value reached was 0.07585714012384415
